In [ ]:
#%matplotlib ipympl
import pykonal
import matplotlib
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd

## Load data

srcs = pd.read_csv('data/sources.dat',delim_whitespace=True,names=['x','z'])
rcs  = pd.read_csv('data/STATIONS',delim_whitespace=True,names=['sname','net','x','z','cc1','cc2'])

# I am using the y-axis as the vertical axis; it is positive downwards.
srcs['y'] = 3e3 - srcs['z']
# Set the z-coordinate = 0 everywhere to keep everything in the xy-plane
srcs['z'] = 0
# Again for receivers...
rcs['y']  = 3e3 - rcs['z']
rcs['z']  = 0

## Calculate travel times

# The source and receiver locations appear to be in units of [m], so I will
# use units of [m] for distance and [m/s] for velocity to stay consistent.
solver = pykonal.EikonalSolver()
solver.vgrid.min_coords     = 0, 0, 0
solver.vgrid.node_intervals = 100, 100, 100 # Node intervals in [m]
solver.vgrid.npts           = 91, 31, 1
solver.pgrid.min_coords     = solver.vgrid.min_coords
solver.pgrid.node_intervals = solver.vgrid.node_intervals
solver.pgrid.npts           = solver.vgrid.npts
solver.vv = np.ones(solver.vgrid.npts)  * 4e3# Velocity in [m/s]

dobs = np.zeros((len(srcs), len(rcs)))

for isrc, src_row in srcs.iterrows():
    src = src_row[['x', 'y', 'z']]
    solver.clear_sources()
    print(src)
    solver.add_source(src)
    solver.solve()
    ui = pykonal.LinearInterpolator3D(solver.pgrid, solver.uu)
    for irc, rc_row in rcs.iterrows():
        rc  = rc_row[['x', 'y', 'z']]
#         print (rc)
# Ray tracing is very slow right now
        ray = solver.trace_ray(rc, tolerance=0.1)
        dobs[isrc,irc] = ui(rc)

## Plotting

def plot(solver):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, aspect=1)
    ax.pcolormesh(
        solver.pgrid[:, :, 0, 0], 
        solver.pgrid[:, :, 0, 1], 
        solver.uu[:, :, 0], 
        cmap=plt.get_cmap('jet_r')
    )
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.invert_yaxis()

plot(solver)

In [ ]:
solver.trace_ray(rc,tolerance=0.1)